## 5 - Convolutional Sequence to Sequence Learning

Trong notebook lần này, chúng ta sẽ implementing model trong paper [Convolutional Sequence to Sequence Learning](https://arxiv.org/pdf/1705.03122.pdf).

![figure1](./images/5.enc_dec.PNG)

### Introduction

> Model trong notebook lần này khác biệt khá nhiều so với models ở các notebook trước. Lần này, chúng ta sẽ không sử dụng RNN để xây dựng model. Thay vào đó, chúng ta sử dụng Convolutional layers, thường được sử dụng trong xử lý ảnh. Tham khảo [link này](https://github.com/longlnOff/sentiment-analysis/blob/main/4.%20Convolutional%20Sentiment%20Analysis.ipynb) để có cái nhìn tổng quan về CNN cũng như cách áp dụng CNN trong bài toán sentiment analysis. Bên cạnh đó, nhấn vào [đây](https://github.com/longlnOff/sentiment-analysis) để tìm hiểu thêm về bài toán sentiment analysis và các phương pháp xử lý.

> Về cơ bản, convolutional layers sử dụng các filters. Các filters này có chiều rộng (width) và chiều cao (height). Nếu filter có width bằng 3, nó có thể quan sát 3 tokens liên tiếp. Mỗi convolutional layer có rất nhiều filter (trong notebook này là 1024 filters). Mỗi filter trượt dọc theo sequence, từ đầu tới cuối và quan sát 3 tokens đúng liền kề nhau ở mỗi lần trượt (trong trường hợp filter có width bằng 3). **Ý tưởng chính ở đây là: Mỗi một filter trong 1024 filters sẽ trích xuất ra đặc tính khác nhau từ văn bản. Kết quả của quá trình trích xuất đặc trưng này sẽ model sử dụng làm đầu vào cho các convolutional layers khác. Nhiều lớp convolutional sẽ trích xuất nhiều đặc trưng từ source sentence để dịch nó sang target sequence.

### Preparing the Data

> Import các module cần thiết và thiết lập random seeds.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Load spacy models và định nghĩa tokenizers cho source sentence và target sentence.

In [ ]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

> Tiếp theo, ta thiết lập `Field`s để xác định cách dữ liệu được xử lý. Mặc định, các RNN models trong Pytorch yêu cầu input sequence là tensor có kích thước **[sequence length, batch size]**, và torchtext cũng mặc định trả về batches các tensor có kích thước tương tự. Tuy nhiên, trong notebook này, chúng ta sử dụng CNNs, và CNNs yêu cầu input là tensor có kích thước **[batch size, sequence length]**. Ta thông báo cho torchtext chuyển tensor mặc định sang tensor có kích thước  **[batch size, sequence length]** bằng cách thiết lập tham số `batch_first = True`.

> Bên cạnh đó, chúng ta cũng sẽ thực hiện chuyển các tokens về lower text và chèn token bắt đầu và token kết thúc sequence.

In [ ]:
SRC = Field(tokenize = tokenize_de,
            init_token='',
            eos_token='',
            lower=True,
            batch_first=True)

TRG = Field(tokenize = tokenize_en,
            init_token='',
            eos_token='',
            lower=True,
            batch_first=True)

> Tiếp theo, ta thực hiện load dataset.

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

> Build Vocabulary, coi các tokens xuất hiện ít hơn 2 lần trong corpus là \<unk> token.

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

> Bước cuối cùng trong công đoạn chuẩn bị dữ liệu là xây dựng iterator.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
train_iterator, valid_iterator, test_iterator = \
        BucketIterator.splits(
            (train_data, valid_data, test_data),
            batch_size=BATCH_SIZE,
            device=device)


### Building the Model

> Tiếp theo, ta thực hiện xây dựng model. Tương tự các notebook trước, model được tạo bởi 2 khối: encoder và decoder. Khối encoder thực hiện encodes input sentence, ở dạng source language, thành *context vector*. Khối decoder sẽ decodes context vector để sinh ra output sentence ở dạng target language.

##### Encoder

> Models ở các notebook trước sử dụng encoder để nén toàn bộ thông tin của input sentence vào một single context vector,z. Tuy nhiên, convolutional seq2seq model có một chút khác biệt - với mỗi token trong input sequence, ta sẽ thu được 2 context vectors. Nếu input sentence có 6 tokens, ta sẽ thu được 12 context vectors (2 vector cho mỗi token). 

> Hai context vector thu được từ mỗi token là *conved* vector và *combined* vector. *Conved* vector là kết quả của mỗi token khi được truyền qua một vài layers. *Combined* vector là tổng của convolved vector và embedding của token đó. Cả 2  vector này được trả về từ encoder để phục vụ cho quá trình decode.

> Hình dưới minh họa việc một input sentence (zwei menschen fetchen) được truyền vào encoder.

![figure2](./images/5.enc_example.PNG)

> Trước tiên, được truyền qua *token embedding layer*. Tuy nhiên, model này không chứa các RNN nên nó không thể nhận biệt được thứ tự của các tokens bên trong sequence. Để khắc phục vấn đề này, ta có một embedding layer thứ 2 - *positional embedding layer*. Đây là một embedding layer chuẩn với đầu vào không phải là tokens, thay vào đó, ta truyền vào vị trí của tokens bên trong sequence - bắt đầu với \<sos> token, ở vị trí 0.

> Tiếp theo, token embeddings và positional embeddings được *elementwise summed* để sinh ra vector chứa thông tin về token cũng như vị trí của token đó bên trong sequence - từ nay ta sẽ gọi vector tổng hợp này là *embedding vector*. Sau đó, ta đưa *embedding vector* này qua một linear layer để biến đối nó thành vector khác có số chiều tương ứng với hidden dimension size mong muốn (đầu ra sẽ là hidden vector).

> Bước tiếp theo, ta cần truyền *hidden vector* này vào **N** *convolutionalblocks*. Chi tiết về hoạt động của *convolutional blocks* sẽ được trình bày kỹ hơn ở phần sau. Sau khi đi qua convolutional blocks, vector sẽ được đưa vào một linear layer khác để chuyển nó từ vector có kích thước bằng hidden dimension size sang embedding dimension size. Quá trình trên tạo ra ***conved*** vector - và mỗi token, ta thu được một ***conved*** vector. 

> Cuối cùng, ***conved*** vector được *elementwise summed* với embedding vector thông qua residual connection để sinh ra ***combined*** vector cho mỗi token. Và mỗi token sẽ có một ***combined*** vector.

> Như vậy, ta thu được 2 vector là: ***conved*** vector và ***combined*** vector đối với mỗi token trong input sequence.

##### Convolutional Blocks

> Phần này sẽ trình bày ngắn gọn hoạt động của convolutional blocks. 

> Hình dưới mô tả hoạt động của 2 convolutional blocks với một filter (blue) trượt ngang qua các tokens bên trong sequence. Trong notebook này, chúng ta sẽ xây dựng encoder với 10 convolutional blocks, mỗi block có 1024 filter.

![figure3](./images/5.convolutional_blocks.PNG)

> Đầu tiên, input sentence được đệm, lý do là convolutional layers sẽ giảm chiều dài của input sentence. Tuy nhiên, ở đây chúng ta mong muốn chiều dài câu đầu vào của convolutional blocks phải bằng với chiều dài output sentence của convolutional blocks. Nếu như không padding, chiều dài output sentence của convolutional blocks sẽ là: "**chiều dài câu đầu vào - `filter_size - 1`**", ngắn hơn sequence được đưa vào convolutional layer `filter_size -1` đơn vị. Ví dụ, filter có size bằng 3 thì output sentence sẽ ngắn hơn input sentence 2 đơn vị. Để khắc phục vấn đề này, ta thực hiện padding sequence ở đầu và cuối, mỗi phía 1 token. Ta có công thức tính toán số token cần đệm ở mỗi phía trong trường hợp filter size là số lẻ như sau: <br>
>> `number padding token each side = (filter_size - 1) / 2` <br>

> (Ở đây, ta không xét trường hợp filter có size là số chẵn).

> Các filter này được thiêt kế sao cho output hidden dimension gấp đôi input hidden dimension của chúng. Trong computer vision, hidden dimensions được gọi là *channels*, tuy nhiên trong NLP ta sẽ gọi chúng là hidden dimensions. 

> Câu hỏi đặt ra ở đây là: tại sao chúng ta lại nhân đôi kích thước đầu ra cua convolutional filter so với đầu vào? Lý do là ở đây, chúng ta sử dụng một activation function đặc biệt có tên là *gated linear units* (GLU). GLUs tương tự như LSTM hay GRU, đều có cơ chế *gating*. GLUs sẽ trả về đầu ra có kích thước giảm một nửa => chúng ta cần nhân đôi đề đầu vào và đầu ra có kích thước bằng nhau.

> Sau khi đi qua GLUs activation function, hidden dimension size của mỗi token là bằng nhau trước và sau khi đi qua convolutional blocks. Tiếp theo, chúng được *elementwise summed* với chính vector của nó trước khi được truyền vào convolutional layer.

> Trên đây là hoạt động của single convlutional block. Các blocks tiếp theo có input là output của previous block và thực hiện công việc tương tự. Mỗi block có một bộ parameters riêng của nó, các bộ parameters này không được shared giữa các blocks. Output của last block sẽ quay trở lại encoder và được đưa vào linear layer để sinh ra *conved* vector, sau đó được *elementwise summed* với embedding của token để sinh ra *combined* vector.

##### Encoder Implementation

> Để đơn giản hóa, ta chỉ sử dụng filter có size là số lẻ - việc này cho phép padding source sentence ở cả 2 phía bằng nhau.

> Biến `scale` được sử dụng bởi tác giả để "đảm bảo phương sai của network không bị thay đổi quá nhiều". Hiệu suất của model sẽ tay đổi khá nhiều nếu sử dụng seed random khác nhau.

> Positional embedding được khởi tạo để xử lý lý "vocabulary" có kích thước bằng 100. Có nghĩa là nó có thể xử lý sequences có chiều dài tối đa 100 tokens, từ vị trí 0 tới 99. Ta có thể tằng kích thước này nếu sequence có chiều dài lớn hơn.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        assert kernel_size % 2 == 1, "Kernel size must be odd!"
        
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(input_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size, 
                                              padding = (kernel_size - 1) // 2)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        
        #src = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        #create position tensor
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [0, 1, 2, 3, ..., src len - 1]
        
        #pos = [batch size, src len]
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(src)
        pos_embedded = self.pos_embedding(pos)
        
        #tok_embedded = pos_embedded = [batch size, src len, emb dim]
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #embedded = [batch size, src len, emb dim]
        
        #pass embedded through linear layer to convert from emb dim to hid dim
        conv_input = self.emb2hid(embedded)
        
        #conv_input = [batch size, src len, hid dim]
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        #conv_input = [batch size, hid dim, src len]
        
        #begin convolutional blocks...
        
        for i, conv in enumerate(self.convs):
            #pass through convolutional layer
            conved = conv(self.dropout(conv_input))

            #conved = [batch size, 2 * hid dim, src len]

            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale

            #conved = [batch size, hid dim, src len]
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
        
        #...end convolutional blocks
        
        #permute and convert back to emb dim
        conved = self.hid2emb(conved.permute(0, 2, 1))
        
        #conved = [batch size, src len, emb dim]
        
        #elementwise sum output (conved) and input (embedded) to be used for attention
        combined = (conved + embedded) * self.scale
        
        #combined = [batch size, src len, emb dim]
        
        return conved, combined

##### Decoder

> Decoder đọc actual target sentence và cố gắng predict nó. Decoder model có sự khác biệt so với RNN models được sử dụng từ trước - lần này, decoder thực hiện predict tất cả các token trong một lần, quá trình predict các tokens được thực hiện song song với nhau. Không có sequential processing, không có decoding loop, ...

> Decoder khá giống với encoder, với một chút thay đổi về convolutional block bên trong model.

![figure4](./images/5.dec.PNG)

> Đầu tiên, embeddings không có residual connection đảm nhận việc kết nối sau convolutional blocks và transformation. Thay vào đó, embedding được truyền vào convolutional blocks để được sử dụng giống như một residual connections.

> Thứ 2, để truyền thông tin từ encoder sang decoder, encoder conved và combined output được sử dụng bên trong convolutional blocks.

> Cuối cùng, output của decoder là một linear layer chuyển từ embedding dimension sang output dimension. Nó được sử dụng để đưa ra dự đoán về từ tiếp theo trong quá trinh dịch.

##### Decoder Convolutional Blocks

> Convolutional blocks bên trong encoder cũng có một vài thay đổi.

![figure5](./images/5.dec_conv.PNG)

> Đầu tiên là padding. Thay vì padding ở 2 phía của sentence, chúng ta chỉ padding vào đầu của sentence. Do chúng ta đang sử dụng đồng thời tất cả các targets một cách song song (không phải tuần tự), nên chúng ta cần một phương pháp cho phép filters dịch token thứ i chỉ được quan sát các tokens nằm trước từ thứ i. Nếu các filters quan sát được từ phía sau, chúng sẽ sao chép các kết quả và không học được gì.

> Cùng quan sát việc đệm ký tự ở hai phía cho kết quả SAI như thế nào:

> ![figure](./images/5.incorrectly.PNG)

> Filter ở vị trí đầu tiên đang sử dụng từ thứ nhất trong sequence (\<sos> token) để dự đoán từ thứ hai (từ two), bây giờ có thể trực tiếp quan sát từ thứ 2 (từ two). Quá trình trên xảy ra ở tất cả các vị trí, từ mà model cố gắng dự đoán là phần từ thứ 2 được chính filter của model quan sát (giống như bạn làm bài tập mà biết trước lời giải => khả năng cao là sẽ không học được gì). Việc quan sát trước kết quả khiến filters chỉ đơn giản là sao chép từ ở kết quả mà không học được gì từ việc dịch.

> Thứ hai, sao GLU activation function và trước residual connection, các khối tính toán được áp dụng attention - sử dụng biểu diễn đã được encoded và embedding của từ hiện tại. 

![figure5](./images/5.dec_conv.PNG)

> **NOTE:** lưu ý là hình trên chỉ thể hiện việc tính attention cho các tokens ở ngoài cùng bên phải, nhưng thực chất, nó được kết với tất cả các tokens. Mỗi token CHỈ sử dụng embedding của chính token đó để tính attention của chính nó.

> Attention được tính bằng cách: đầu tiên, sử dụng một linear layer để thay đổi hidden dimension sao cho có cùng kích thước với embedding dimension. Sau đó cộng embedding bằng residual connection.  This combination then has the standard attention calculation applied by finding how much it "matches" with the encoded conved and then this is applied by getting a weighted sum over the encoded combined. This is then projected back up to the hidden dimenson size and a residual connection to the initial input to the attention layer is applied.

> Vậy tại sao chúng ta cần tính attention trước với encoded conved và sau đó sử dụng nó để tính tổng trọng số thông qua việc kết hợp? Dưới đây là 2 vấn đề mà paper đang cân nhắc: <br>
>> * encoded conved rất tốt cho việc lấy bối cảnh tổng quan thông qua encoded sequence.
>> * Trong khi đó, encoded combined chứa nhiều thông tin về một token cụ thể hơn, từ đó rất phù hợp cho việc dự đoán.

##### Decoder Implementation

> Do trong bài này, chúng ta chỉ thực hiện padding một phía nên decoder được cho phép sử dụng padding có kích thước lẻ hoặc chẵn. Và, `scale` được sử dụng để giảm phương sai của model. Position embedding được khởi tạo với kích thước vocabulary bằng 100.

> Model nhận vào biểu diễn của encoder trong `forward` method, và cả hai được áp dụng `calculate_attention` để tính và áp dụng attention. Model có trả về giá trị attention nhưng ở dây, ta không sử dụng chúng.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 trg_pad_idx, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.kernel_size = kernel_size
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(output_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.attn_hid2emb = nn.Linear(hid_dim, emb_dim)
        self.attn_emb2hid = nn.Linear(emb_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim, output_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
      
    def calculate_attention(self, embedded, conved, encoder_conved, encoder_combined):
        
        #embedded = [batch size, trg len, emb dim]
        #conved = [batch size, hid dim, trg len]
        #encoder_conved = encoder_combined = [batch size, src len, emb dim]
        
        #permute and convert back to emb dim
        conved_emb = self.attn_hid2emb(conved.permute(0, 2, 1))
        
        #conved_emb = [batch size, trg len, emb dim]
        
        combined = (conved_emb + embedded) * self.scale
        
        #combined = [batch size, trg len, emb dim]
                
        energy = torch.matmul(combined, encoder_conved.permute(0, 2, 1))
        
        #energy = [batch size, trg len, src len]
        
        attention = F.softmax(energy, dim=2)
        
        #attention = [batch size, trg len, src len]
            
        attended_encoding = torch.matmul(attention, encoder_combined)
        
        #attended_encoding = [batch size, trg len, emd dim]
        
        #convert from emb dim -> hid dim
        attended_encoding = self.attn_emb2hid(attended_encoding)
        
        #attended_encoding = [batch size, trg len, hid dim]
        
        #apply residual connection
        attended_combined = (conved + attended_encoding.permute(0, 2, 1)) * self.scale
        
        #attended_combined = [batch size, hid dim, trg len]
        
        return attention, attended_combined
        
    def forward(self, trg, encoder_conved, encoder_combined):
        
        #trg = [batch size, trg len]
        #encoder_conved = encoder_combined = [batch size, src len, emb dim]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
            
        #create position tensor
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, trg len]
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(trg)
        pos_embedded = self.pos_embedding(pos)
        
        #tok_embedded = [batch size, trg len, emb dim]
        #pos_embedded = [batch size, trg len, emb dim]
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #embedded = [batch size, trg len, emb dim]
        
        #pass embedded through linear layer to go through emb dim -> hid dim
        conv_input = self.emb2hid(embedded)
        
        #conv_input = [batch size, trg len, hid dim]
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        #conv_input = [batch size, hid dim, trg len]
        
        batch_size = conv_input.shape[0]
        hid_dim = conv_input.shape[1]
        
        for i, conv in enumerate(self.convs):
        
            #apply dropout
            conv_input = self.dropout(conv_input)
        
            #need to pad so decoder can't "cheat"
            padding = torch.zeros(batch_size, 
                                  hid_dim, 
                                  self.kernel_size - 1).fill_(self.trg_pad_idx).to(self.device)
                
            padded_conv_input = torch.cat((padding, conv_input), dim = 2)
        
            #padded_conv_input = [batch size, hid dim, trg len + kernel size - 1]
        
            #pass through convolutional layer
            conved = conv(padded_conv_input)

            #conved = [batch size, 2 * hid dim, trg len]
            
            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, trg len]
            
            #calculate attention
            attention, conved = self.calculate_attention(embedded, 
                                                         conved, 
                                                         encoder_conved, 
                                                         encoder_combined)
            
            #attention = [batch size, trg len, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale
            
            #conved = [batch size, hid dim, trg len]
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
            
        conved = self.hid2emb(conved.permute(0, 2, 1))
         
        #conved = [batch size, trg len, emb dim]
            
        output = self.fc_out(self.dropout(conved))
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

##### Seq2Seq

> Module đóng gói, `Seq2Seq` có khá nhiều khác biệt so với module Seq2Seq khi áp dụng RNN, đặc biệt là phần decoding.

> Biến `trg` có phần tử \<eos> được cắt bỏ ở cuối do chúng ta không truyền \<eos> token vào decoder.

> Quá trình encoding khá giống với các notebook trước, đưa input sentence và nhận lại context vector. Tuy nhiên ở đây chúng ta có tới 2 context vector cho mỗi từ trong source sequence, `encoder_conved` và `encoder_combined`.

> Quá trình decoding được thực hiện song song, do đó chúng ta không cần decoding loop. Tất cả các target sequence đều được đưa vào decoder cùng một lúc và việc padding được sử dụng để đảm bảo mỗi convolutional filter trong decoder chỉ có thể quan sát token hiện tại và token phía trước khi nó trượt trong sentence.

> Điều này có nghĩa là chúng ta không áp dụng teacher forcing trên model này. Chúng ta không cần vòng lặp khi predict.

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len - 1] ( token sliced off the end)
           
        #calculate z^u (encoder_conved) and (z^u + e) (encoder_combined)
        #encoder_conved is output from final encoder conv. block
        #encoder_combined is encoder_conved plus (elementwise) src embedding plus 
        #  positional embeddings 
        encoder_conved, encoder_combined = self.encoder(src)
            
        #encoder_conved = [batch size, src len, emb dim]
        #encoder_combined = [batch size, src len, emb dim]
        
        #calculate predictions of next words
        #output is a batch of predictions for each word in the trg sentence
        #attention a batch of attention scores across the src sentence for 
        #  each word in the trg sentence
        output, attention = self.decoder(trg, encoder_conved, encoder_combined)
        
        #output = [batch size, trg len - 1, output dim]
        #attention = [batch size, trg len - 1, src len]
        
        return output, attention

##### Training the Seq2Seq Model

> Phần còn lại khá giống với các notebook trước, ta thực hiện define các hyperparameters, khởi tạo encoder và decoder, ...

> Trong paper, người ta đã khảo sát và thấy rằng việc sử dụng nhiều filter có kích thước nhỏ cho kết quả tốt hơn (ở đây dùng nhiều hơn 5 filter, mỗi filter có kích thước bằng 3).

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
EMB_DIM = 256
HID_DIM = 512 # each conv. layer has 2 * hid_dim filters
ENC_LAYERS = 10 # number of conv. blocks in encoder
DEC_LAYERS = 10 # number of conv. blocks in decoder
ENC_KERNEL_SIZE = 3 # must be odd!
DEC_KERNEL_SIZE = 3 # can be even or odd
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
    
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, ENC_LAYERS, ENC_KERNEL_SIZE, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, DEC_LAYERS, DEC_KERNEL_SIZE, DEC_DROPOUT, TRG_PAD_IDX, device)

model = Seq2Seq(enc, dec).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

> Tiếp theo, ta định nghĩa optimizer và loss function. Giống các notebook trước, chúng ta ignore loss khi target sequence là padding token.

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

> Sau đó là training loop cho model.

> Chúng ta xử lý sequences hơi khác so với các notebook trước. Với các model dùng RNN, chúng ta không bao giờ đưa \<eos> vào decoder. Trong model này, chúng ta chỉ đon giản loại bỏ \<eos> trong sequence. Do đó: <br>
>>> trg = [sos, $x_1$, $x_2$, $x_3$, eos] <br>
>>> trg[:-1] = [sos, $x_1$, $x_2$, $x_3$]

> Ở đây, $x_i$ là các phần tử trong target sequence. Chúng ta sẽ đưa chúng vào trong model để dự đoán: <br>
>>> output = [$y_1$, $y_2$, $y_3$, eos]

> $y_i$ là các phần tử trong predicted sequence. Ta tính toán loss thông qua tensor gốc `trg` sau khi loại bỏ \<sos> token, để lại \<eos> token: <br>
>>> output = [$y_1$, $y_2$, $y_3$, eos] <br>
>>> trg[1:] = [$x_1$, $x_2$, $x_3$, eos]

> Chúng ta thực hiện tính loss và cập nhật tham số như bình thường.

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
        
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
        
        output_dim = output.shape[-1]
        
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

> Evaluation loop.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
        
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]

            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

> Hàm tính thời gian.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 0.1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
model.load_state_dict(torch.load('./models/tut5-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

##### Inference

> Hàm `translate_sentence` sẽ thực hiện các công việc sau:
>> * Đảm bảo model ở evaluation mode (inference thì model luôn luôn phải ở chế độ evaluation).
>> * Tokenize source sentece nếu nó chưa được tokenized.
>> * Chèn \<sos> và \<eos> tokens.
>> * Số hóa source sentence.
>> * Convert source sentence đã được số hóa thành tensor và thêm vào batch dimension.
>> * Đưa source sentence vào encoder.
>> * Tạo list để lưu trữ output sentence, được khởi tạo với một token \<sos>.
>> * Lặp khi chưa đạt đến độ dài tối đa:
>>>> * convert câu dự đoán hiện tại thành tensor với batch dimension.
>>>> * Đưa current output và 2 outputs từ encoder vào decoder.
>>>> * Lấy giá trị dự đoán của token tiếp theo.
>>>> * Thêm giá trị dự đoán vào output sentence prediction.
>>>> * break nếu prediction là \<eos> token.
> * Chuyển output sentence từ indexes sang tokens.
> * Trả về output sentence (sau khi đã removed các \<sos> token) và attention value của mỗi sequence.

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    with torch.no_grad():
        encoder_conved, encoder_combined = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, encoder_conved, encoder_combined)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

>Hàm hiển thị attention.

In [ ]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
        
    attention = attention.squeeze(0).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    ax.set_xticklabels(['']+['']+[t.lower() for t in sentence]+[''], 
                       rotation=45)
    ax.set_yticklabels(['']+translation)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

> Bắt đầu dịch, dưới đây là  example trong training set.

In [ ]:
example_idx = 2

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

In [ ]:
display_attention(src, translation, attention)

### BLEU

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off  token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

> Đây là notebook đầu tiên chúng ta không sử dụng RNN. Trong phần tiếp theo, chúng ta cùng tìm hiểu `Transformer` model - model này thậm chí còn không dùng đến convolutional layers, chỉ dùng các linear layers và rất rất nhiều attention mechanisms.

## END